# Email Bot Notebook

This notebook demonstrates how to:
1. Authenticate with Gmail's API.
2. Retrieve unread emails.
3. Summarize or filter them (optionally).
4. Send a daily recap email back to yourself.
5. Mark processed emails as read.

We'll use the [Google API Python Client](https://github.com/googleapis/google-api-python-client) for Gmail access.


## 1. Setup and Installation

Make sure to install the necessary packages in your environment:

In [ ]:
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

## 2. Imports and Configuration

We'll import the libraries we need and set up any global constants such as:
- `SCOPES`: Gmail API scopes
- `CREDENTIALS_PATH`: Path to our `credentials.json`
- `TOKEN_PATH`: Path to store the OAuth tokens (so we don’t have to re-authenticate every time).


In [1]:
import os
import base64
import datetime

from email.mime.text import MIMEText

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

# If modifying these scopes, delete your existing token file.
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

# Paths to your files
CREDENTIALS_PATH = "secrets/credentials.json"  # Adjust as needed
TOKEN_PATH = "secrets/token.pickle"  # Where the OAuth tokens will be stored

## 3. Authentication

We'll do the following:
1. Check if we already have a valid `token.pickle`.
2. If not, we prompt the user to log in via OAuth.
3. We then save the token for future sessions.


In [3]:
def get_gmail_service():
    creds = None

    # 1. Check if token.pickle exists
    if os.path.exists(TOKEN_PATH):
        with open(TOKEN_PATH, "rb") as token_file:
            creds = pickle.load(token_file)

    # 2. If no valid creds or they are expired, go through the OAuth flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
            creds = flow.run_local_server(port=0)

        # 3. Save the credentials
        with open(TOKEN_PATH, "wb") as token_file:
            pickle.dump(creds, token_file)

    # 4. Build the Gmail service
    service = build("gmail", "v1", credentials=creds)
    return service


# Test it once
service = get_gmail_service()
print("Gmail service created successfully.")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=862123276762-8nq37c8j6vqp9ntrs1lkv11sf180ahel.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51404%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=gNQ6wacP9cv3zPO3kAlUdkkn1B0Mvb&access_type=offline
Gmail service created successfully.


## 4. Retrieving Unread Emails

We'll fetch a list of messages labeled `UNREAD`, retrieve their details, and store them for processing.


In [5]:
def get_unread_emails_past_24h(service, max_results=10):
    """
    Fetch unread emails from the last 24 hours.
    """
    try:
        # Use a query that searches for emails unread and newer than 1 day
        results = (
            service.users()
            .messages()
            .list(userId="me", q="is:unread newer_than:1d", maxResults=max_results)
            .execute()
        )

        messages = results.get("messages", [])

        emails_data = []
        for msg in messages:
            # Get the message details
            msg_detail = (
                service.users().messages().get(userId="me", id=msg["id"]).execute()
            )

            subject = None
            snippet = msg_detail.get("snippet", "")
            headers = msg_detail.get("payload", {}).get("headers", [])
            for header in headers:
                if header["name"].lower() == "subject":
                    subject = header["value"]
                    break

            emails_data.append(
                {"id": msg["id"], "subject": subject, "snippet": snippet}
            )

        return emails_data

    except Exception as error:
        print(f"An error occurred: {error}")
        return []


unread_emails = get_unread_emails_past_24h(service)
print(number_of_unread_emails := len(unread_emails), "unread emails found.")

10 unread emails found.


## 5. Import and Configure OpenAI to use for Sorting, Summarizing Emails, as well as Generating the report. 

In [38]:
import os
import json
import openai

# Get API key from `secrets/openai.json`
with open("secrets/openai.json") as f:
    data = json.load(f)
    openai.api_key = data["api_key"]

## 7. Classification Function
This function sends the email’s subject and snippet to GPT and expects a JSON response with a single category field (e.g. {"category": "newsletter"}).

In [17]:
import json
import openai

CLASSIFY_AND_SUMMARIZE_FUNCTION = {
    "name": "classify_and_summarize",
    "description": (
        "Classify an email into exactly one category: "
        "newsletter, notification, or important. Also provide "
        "a short, neatly formatted Markdown summary."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "category": {
                "type": "string",
                "description": (
                    "Which category the email belongs to. Must be exactly one of:\n\n"
                    "1) **newsletter** - If the email is from a mailing list, "
                    "contains marketing/promotional content, or is part of a "
                    "regularly scheduled mass distribution (e.g., weekly digest).\n\n"
                    "2) **notification** - If the email is an automated alert, "
                    "shipping update, system message, or other non-personal, "
                    "status-based communication.\n\n"
                    "3) **important** - If the email is personal, urgent, or "
                    "from a human sender discussing timely or critical matters, "
                    "such as business proposals, direct communications, or anything "
                    "requiring personal attention."
                ),
                "enum": ["newsletter", "notification", "important"],
            },
            "summary": {
                "type": "string",
                "description": (
                    "A concise Markdown summary of the key points from the email. "
                    "Include relevant next steps or instructions if the email references "
                    "new invitations, messages, or any call to action. Label this section "
                    "as 'Next Steps' or 'Actions' in bullet points if applicable. "
                    "The summary should be neatly formatted in MD and not omit essential details."
                ),
            },
        },
        "required": ["category", "summary"],
        "additionalProperties": False,
    },
}


def classify_and_summarize_email(subject: str, snippet: str) -> dict:
    """
    Uses OpenAI's function calling to:
      - Classify the email as 'newsletter', 'notification', or 'important'.
      - Provide a Markdown summary capturing key points from the email.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are an email classification and summarization service. "
                "Categorize the email into exactly one of: 'newsletter', 'notification', or 'important'. "
                "Then produce a concise summary in Markdown, carefully extracting key points without "
                "omitting vital information.\n\n"
                "If the email references new invitations, messages, or calls to action:\n"
                "- **If** the snippet or subject **mentions** a specific platform (e.g. LinkedIn), "
                "include that detail in the summary.\n"
                "- **If** it doesn’t specify the platform, explicitly state that it’s not mentioned.\n\n"
                "### Example\n"
                "#### Email Body:\n"
                '"You have 3 new invitations on LinkedIn. Review them as soon as possible!"\n\n'
                "**Correct Summary (MD):**\n"
                "```\n"
                "### New Invitations\n"
                "You have 3 new invitations on LinkedIn.\n\n"
                "**Next Steps:**\n"
                "- Log into your LinkedIn account to view and respond.\n"
                "```\n\n"
                "#### Another Email Body:\n"
                '"You have 3 new invitations!"\n\n'
                "**Correct Summary (MD):**\n"
                "```\n"
                "### New Invitations\n"
                "You have 3 invitations.\n\n"
                "**Next Steps:**\n"
                "- The email doesn’t specify a platform or link. Possibly log in to your most-used accounts, "
                "  or contact the sender for further details.\n"
                "```\n\n"
                "Only use details that appear in the email subject/snippet. Do not invent or guess any details "
                "if they are not explicitly mentioned."
            ),
        },
        {
            "role": "user",
            "content": f"Subject: {subject}\n\nEmail snippet/body:\n{snippet}",
        },
    ]

    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            functions=[CLASSIFY_AND_SUMMARIZE_FUNCTION],
            function_call={"name": "classify_and_summarize"},  # Force the function call
        )

        choice = response.choices[0]

        # Access function_call as a property
        function_call_obj = choice.message.function_call
        if not function_call_obj:
            print("[DEBUG] No function_call object found. Falling back.")
            return {
                "category": "important",
                "summary": (
                    "Unable to parse email with function calling. "
                    "Defaulting to 'important'."
                ),
            }

        arguments_str = function_call_obj.arguments
        print("[DEBUG] function_call.arguments =>", arguments_str)

        args_dict = json.loads(arguments_str)
        category = args_dict.get("category", "important")
        summary_md = args_dict.get("summary", "")

        return {"category": category, "summary": summary_md}

    except Exception as e:
        print("Error during classification/summarization:", e)
        return {
            "category": "important",
            "summary": (
                "Error occurred. Defaulting to category 'important'.\n\n"
                f"**Details:** {str(e)}"
            ),
        }


classified_emails = []

for idx, email in enumerate(unread_emails, start=1):
    subject = email["subject"] or "(No Subject)"
    snippet = email["snippet"] or ""

    print(f"\n=== Email #{idx} ===")
    result = classify_and_summarize_email(subject, snippet)

    email_classification = result["category"]
    email_summary_md = result["summary"]

    # Store in a list or attach to the original email dict
    classified_emails.append(
        {
            "id": email["id"],
            "subject": subject,
            "snippet": snippet,
            "category": email_classification,
            "summary": email_summary_md,
        }
    )

    # Print a quick summary
    print("Classification:", email_classification)
    print("Summary (MD):", email_summary_md)


=== Email #1 ===
[DEBUG] function_call.arguments => {"category":"notification","summary":"### New Sign-in Notification\n\nYour Login.gov email and password were used to sign-in and authenticate on a new device.\n\n**Actions:**\n- If you recognize this activity, no further action is required.\n- If you do not recognize this activity, consider checking your account security settings or contacting support."}
Classification: notification
Summary (MD): ### New Sign-in Notification

Your Login.gov email and password were used to sign-in and authenticate on a new device.

**Actions:**
- If you recognize this activity, no further action is required.
- If you do not recognize this activity, consider checking your account security settings or contacting support.

=== Email #2 ===
[DEBUG] function_call.arguments => {"category":"newsletter","summary":"### Weekly Trailhead Military Digest\nThis is a weekly digest for the 'Become a Salesforce Certified Admin' program.\n\n**Date:** December 29, 2024

## 8. Mark UnImportant Emails as Read

In [19]:
# ===========================
# 8. Mark Non-Important Emails as Read
# ===========================


def mark_non_important_as_read(service, classified_emails):
    """
    Iterates over 'classified_emails' and marks any email with
    category != 'important' as 'READ' in Gmail.
    """
    for email in classified_emails:
        category = email.get("category", "important")
        message_id = email.get("id")

        # Skip if there's no valid message ID
        if not message_id:
            continue

        # If category is 'newsletter' or 'notification', mark it as read
        if category != "important":
            try:
                service.users().messages().modify(
                    userId="me", id=message_id, body={"removeLabelIds": ["UNREAD"]}
                ).execute()
                print(f"[DEBUG] Marked '{message_id}' as read (category: {category}).")
            except Exception as e:
                print(f"[ERROR] Could not mark '{message_id}' as read: {e}")


# Now call the function with your existing 'classified_emails' list:
mark_non_important_as_read(service, classified_emails)

[DEBUG] Marked '194366466afd41e1' as read (category: notification).
[DEBUG] Marked '19435a42d221ae48' as read (category: newsletter).
[DEBUG] Marked '194346322c64c487' as read (category: notification).
[DEBUG] Marked '19433e0b5cb53479' as read (category: notification).
[DEBUG] Marked '194338e0c7f268b6' as read (category: newsletter).
[DEBUG] Marked '194331c44d63775b' as read (category: notification).
[DEBUG] Marked '19432ed9b9af8b12' as read (category: newsletter).
[DEBUG] Marked '19432e3baac16952' as read (category: notification).
[DEBUG] Marked '19432dda99595b32' as read (category: newsletter).
[DEBUG] Marked '194326bc87205e65' as read (category: newsletter).


## 9. Executive-Level Recap Generator (Using O1 Model)

This cell introduces a new function, **`generate_executive_summary_o1`**, which calls the O1 model to transform our `classified_emails` data into an intelligence-briefing–style summary. Rather than just listing out each email individually, this summary aims to provide a **cohesive, high-level analysis** of the day’s messages, suitable for a military, corporate, or political leader.

**Key Points**:

- Merges data from all emails into a single “briefing document.”  
- Highlights urgent or critical matters first.  
- Provides context or recommended actions if relevant.  
- Maintains a formal tone appropriate for an “executive-level” audience.


In [28]:
def categorize_emails(classified_emails):
    """
    Assigns emails to logical categories based on content and keywords to create a more
    cohesive, actionable structure. Categories include:
      1. Priority Alerts
      2. Key Operational Items
      3. Additional Notifications
      4. Promotional & Informational
    """

    priority_alerts = []
    key_operational = []
    additional_notifications = []
    promos_informational = []

    for email in classified_emails:
        cat = email.get("category", "").lower()
        subj = email.get("subject", "").lower()
        summ = email.get("summary", "").lower()

        # Create a link to open the email in Gmail (if we have an id)
        msg_id = email.get("id")
        if msg_id:
            email["web_link"] = f"https://mail.google.com/mail/u/0/#inbox/{msg_id}"
        else:
            email["web_link"] = "#"

        # Simple heuristics to demonstrate grouping logic:
        if (
            "sign-in" in summ
            or "security" in summ
            or ("new" in summ and "device" in summ)
        ):
            # Potential security or sign-in alerts
            priority_alerts.append(email)
        elif "monthly statement" in summ or "release" in summ or "delivery" in summ:
            # Operational or time-sensitive items
            key_operational.append(email)
        elif "invitation" in summ:
            additional_notifications.append(email)
        elif (
            cat == "newsletter"
            or "discount" in summ
            or "resolutions" in summ
            or "digest" in summ
        ):
            promos_informational.append(email)
        else:
            # If it doesn't match any of the above heuristics, put it in additional notifications
            additional_notifications.append(email)

    return {
        "priority_alerts": priority_alerts,
        "key_operational": key_operational,
        "additional_notifications": additional_notifications,
        "promos_informational": promos_informational,
    }


def build_briefing_text(email_groups):
    """
    Builds a more structured, analytical briefing text using the grouped emails.
    Incorporates recommendations and potential implications for each category.
    """
    lines = []

    # Header / Intro
    lines.append("**Intelligence Briefing: Executive Summary**")
    lines.append(
        f"**Date:** {get_current_date_string()}\n"
    )  # Helper for date if desired

    # Priority Alerts
    if email_groups["priority_alerts"]:
        lines.append("### 1. Priority Alerts")
        for email in email_groups["priority_alerts"]:
            subj = email.get("subject", "(No Subject)")
            summ = email.get("summary", "(No Summary)")
            link = email.get("web_link", "#")  # Link to open the email in Gmail

            lines.append(f"**Alert:** {subj}")
            lines.append(f"- **Context:** {summ}")
            lines.append(
                "- **Implications:** Potential risk of unauthorized access or urgent action needed."
            )
            lines.append(
                "- **Recommended Action:** Verify legitimacy and secure accounts immediately."
            )
            lines.append(f"- [View Email]({link})\n")

    # Key Operational Items
    if email_groups["key_operational"]:
        lines.append("### 2. Key Operational Items")
        for email in email_groups["key_operational"]:
            subj = email.get("subject", "(No Subject)")
            summ = email.get("summary", "(No Summary)")
            link = email.get("web_link", "#")

            lines.append(f"**Item:** {subj}")
            lines.append(f"- **Context:** {summ}")
            lines.append("- **Impact:** May affect workflow, scheduling, or finances.")
            lines.append(
                "- **Recommended Action:** Review details, coordinate with relevant teams."
            )
            lines.append(f"- [View Email]({link})\n")

    # Additional Notifications
    if email_groups["additional_notifications"]:
        lines.append("### 3. Additional Notifications")
        for email in email_groups["additional_notifications"]:
            subj = email.get("subject", "(No Subject)")
            summ = email.get("summary", "(No Summary)")
            link = email.get("web_link", "#")

            lines.append(f"**Notification:** {subj}")
            lines.append(f"- **Context:** {summ}")
            lines.append(
                "- **Recommended Action:** Follow up if relevant to ongoing projects."
            )
            lines.append(f"- [View Email]({link})\n")

    # Promotional & Informational
    if email_groups["promos_informational"]:
        lines.append("### 4. Promotional & Informational")
        for email in email_groups["promos_informational"]:
            subj = email.get("subject", "(No Subject)")
            summ = email.get("summary", "(No Summary)")
            link = email.get("web_link", "#")

            lines.append(f"**Offer/Info:** {subj}")
            lines.append(f"- **Context:** {summ}")
            lines.append(
                "- **Recommended Action:** Evaluate relevance to personal or organizational goals."
            )
            lines.append(f"- [View Email]({link})\n")

    # Conclusion / Next Steps
    lines.append("### Conclusion & Next Steps")
    lines.append(
        "- Ensure security protocols are followed for any sign-in or account-related alerts."
    )
    lines.append(
        "- Prioritize reviewing new releases or statements affecting operational continuity."
    )
    lines.append(
        "- Address pending invitations or notifications to maintain workflow integrity."
    )
    lines.append(
        "- Evaluate promotional and newsletter content for potential benefits or opportunities.\n"
    )
    lines.append("**End of Briefing**")

    return "\n".join(lines)


def get_current_date_string():
    """Simple helper to return a placeholder or real date string."""
    # For demonstration, you can return a static or dynamically generated date.
    # E.g., using datetime:
    # from datetime import datetime
    # return datetime.now().strftime("%B %d, %Y")
    return "January 5, 2025"


def generate_executive_summary_o1(classified_emails):
    """
    Generates an executive-level briefing from the given list of classified emails,
    using an O1 model that does not support 'system' role.
    Incorporates improvements to create a more cohesive, prioritized, and action-oriented summary.
    """

    # Group emails using our custom categorization logic
    email_groups = categorize_emails(classified_emails)

    # Build the final briefing text
    briefing_text = build_briefing_text(email_groups)

    # Prepare the user message (for the O1 model) with instructions and the structured summary
    user_message = (
        "You are an advanced AI system acting as a top-level intelligence briefer.\n\n"
        "Task:\n"
        "1. Take today's classified emails (some important, some notifications, some newsletters).\n"
        "2. Provide a cohesive, concise summary in the style of an intelligence briefing for a "
        "military, corporate, or political leader.\n\n"
        "You MUST incorporate relevant details from the provided email texts, especially any "
        "urgent calls to action or critical info.\n\n"
        "Format the final result in a clear, structured manner. DO NOT invent details not provided.\n\n"
        "===== DRAFTED BRIEFING =====\n"
        f"{briefing_text}\n"
        "===== END OF DRAFTED BRIEFING =====\n\n"
        "Please refine or confirm the executive-level summary now."
    )

    try:
        response = openai.chat.completions.create(
            model="o1-preview",
            messages=[{"role": "user", "content": user_message}],
            max_completion_tokens=32768,
        )
        executive_summary = response.choices[0].message.content.strip()
        return executive_summary
    except Exception as e:
        print("Error during executive-level summary generation:", e)
        return (
            "An error occurred while generating the executive-level summary.\n\n"
            f"Details: {str(e)}"
        )


summary_report

# Example usage
if __name__ == "__main__":
    # Suppose 'classified_emails' is already defined or loaded
    summary_report = generate_executive_summary_o1(classified_emails)
    print("=== Executive-Level Recap ===\n")
    print(summary_report)

=== Executive-Level Recap ===

**Intelligence Briefing: Executive Summary**

**Date:** January 5, 2025

---

### 1. Priority Alerts

#### **A. Security Alert: New Sign-In on Login.gov Account**

- **Summary:** A new sign-in and authentication have been detected on your Login.gov account from a new device.
- **Actions Required:**
  - **If recognized:** No action needed.
  - **If unrecognized:** Immediately check your account security settings and contact support.
- **Implications:** Potential risk of unauthorized access.
- **Recommended Action:** Verify the sign-in activity and secure your account if necessary.
- [View Email](https://mail.google.com/mail/u/0/#inbox/194366466afd41e1)

---

### 2. Key Operational Updates

#### **A. Financial Statement Available**

- **Summary:** Your monthly statement from Bright Money is now available.
- **Actions Required:**
  - Log in to the Bright Money app.
  - Navigate to `Profile -> Account Info -> Account Statements` to view your statement.
- **Im

### 10. Report Beautification

This cell takes our existing `summary_report` (generated in Markdown) and a basic HTML template, then sends both to GPT-4. The model will convert the Markdown into HTML and inject it into the template at a designated placeholder (e.g., `<!-- REPLACE_ME_WITH_CONTENT -->`). Afterward, we simply display the final HTML so you can review the fully rendered briefing.


In [35]:
# === 10. Report Beautification ===
#
# This cell takes our existing `summary_report` (generated in Markdown) and a basic HTML template,
# then sends both to GPT-4. The model will convert the Markdown into HTML and inject it into the
# template at a designated placeholder (e.g., <!-- REPLACE_ME_WITH_CONTENT -->). Afterward,
# we simply display the final HTML so you can review the fully rendered briefing.

basic_html_template = """\
<html>
<head>
  <meta charset="UTF-8">
  <title>Beautified Executive Summary</title>
  <style>
    body {
      font-family: Arial, sans-serif; 
      margin: 20px;
    }
    h1, h2, h3 {
      color: #333;
    }
    p, li {
      line-height: 1.4em;
    }
    a {
      color: #1976D2; 
      text-decoration: none;
    }
    a:hover {
      text-decoration: underline;
    }
    /* Add more styling as desired */
  </style>
</head>
<body>
  <!-- REPLACE_ME_WITH_CONTENT -->
</body>
</html>
"""


def beautify_md_with_template(md_summary, html_template):
    """
    Sends Markdown summary + HTML template to GPT-4 for conversion.
    GPT-4 will:
    1) Convert the MD summary into HTML.
    2) Inject it into the template (replacing the placeholder).
    3) Return the final HTML string.
    """
    user_prompt = f"""You are a helpful assistant that can convert Markdown to HTML.
We have a basic HTML template with a placeholder: <!-- REPLACE_ME_WITH_CONTENT -->.
We also have a Markdown summary. Insert the rendered HTML from the Markdown summary 
into the template in place of the placeholder, and return the final HTML.

MARKDOWN SUMMARY:
```
{md_summary}
```

HTML TEMPLATE:
```
{html_template}
```

Instructions:
1) Convert the above Markdown into valid HTML.
2) Find <!-- REPLACE_ME_WITH_CONTENT --> in the provided template.
3) Replace that placeholder comment with the converted HTML.
4) Return the resulting fully formed HTML document (only the HTML, no extra commentary).
"""

    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": user_prompt}],
            temperature=0.0,
        )
        final_html = response.choices[0].message.content.strip()
        # Remove first and last lines (``` code fences)
        final_html = "\n".join(final_html.split("\n")[1:-1])
        return final_html
    except Exception as e:
        print("Error converting Markdown to HTML via GPT-4:", e)
        return "<p>An error occurred while generating the final HTML.</p>"


print("md_summary:", summary_report)
# Using the existing `summary_report` as our Markdown summary:
final_html = beautify_md_with_template(summary_report, basic_html_template)

# Print the final HTML output in the notebook
print("=== Final Beautified HTML ===\n")
print(final_html)

md_summary: **Intelligence Briefing: Executive Summary**

**Date:** January 5, 2025

---

### 1. Priority Alerts

#### **A. Security Alert: New Sign-In on Login.gov Account**

- **Summary:** A new sign-in and authentication have been detected on your Login.gov account from a new device.
- **Actions Required:**
  - **If recognized:** No action needed.
  - **If unrecognized:** Immediately check your account security settings and contact support.
- **Implications:** Potential risk of unauthorized access.
- **Recommended Action:** Verify the sign-in activity and secure your account if necessary.
- [View Email](https://mail.google.com/mail/u/0/#inbox/194366466afd41e1)

---

### 2. Key Operational Updates

#### **A. Financial Statement Available**

- **Summary:** Your monthly statement from Bright Money is now available.
- **Actions Required:**
  - Log in to the Bright Money app.
  - Navigate to `Profile -> Account Info -> Account Statements` to view your statement.
- **Implications:** May i

In [36]:
from IPython.display import display, HTML


display(HTML(final_html))

In [37]:
import base64
from email.mime.text import MIMEText
from datetime import datetime


def send_html_email(service, to_address, subject, html_content):
    """
    Sends an HTML email via the provided Gmail service.

    :param service: Authenticated Gmail API service instance.
    :param to_address: Recipient's email address (e.g., your own).
    :param subject: Subject line for the email.
    :param html_content: The final HTML body to send.
    """
    # 1. Create a MIMEText object with 'html' subtype
    message = MIMEText(html_content, "html")
    message["to"] = to_address
    message["subject"] = subject

    # 2. Base64-url-safe encode the message
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode("utf-8")

    body = {"raw": raw_message}

    # 3. Use the Gmail API to send
    try:
        sent_msg = service.users().messages().send(userId="me", body=body).execute()
        print("Email sent successfully! Message ID:", sent_msg["id"])
    except Exception as e:
        print("An error occurred while sending email:", e)


if __name__ == "__main__":
    # final_html = (the HTML from your beautification step)
    # service = (your authenticated Gmail service)

    send_html_email(
        service=service,
        to_address="blgottshall@gmail.com",
        subject=f"AI Email Summary Report {get_current_date_string()}",
        html_content=final_html,
    )

Email sent successfully! Message ID: 19436c9e397b3b04
